In [ ]:
using CairoMakie 
using Arya

using LilGuys

In [ ]:
using PythonCall
agama = pyimport("agama")

In [ ]:
include("../../../notebooks/orbits/agama_utils.jl")


Min(1) / Min(2) = rho_mean1 * r^3 / rho_mean2 * rh^3

In [ ]:
potential_dir = ENV["DWARFS_ROOT"] * "/agama/potentials/"

In [ ]:
readdir(potential_dir )

In [ ]:
scl_profile = NFW(r_circ_max=1.84, v_circ_max=25.75/V2KMS)

In [ ]:
pot_lmc_2 = agama.Potential("$potential_dir/vasiliev24/L2M10/pot/lmc000.ini")

In [ ]:
pot_lmc_3 = agama.Potential("$potential_dir/vasiliev24/L3M10/pot/lmc000.ini")

In [ ]:
pot_lmc_1_5 = agama.Potential("$potential_dir/vasiliev+21/lmc00.pot")

In [ ]:
pot_MW_9_5 = agama.Potential("$potential_dir/vasiliev+21/potential_nolmc.ini")

In [ ]:
pot_MW_10 = agama.Potential("$potential_dir/vasiliev24/L2M10/potential_mw_init.ini")

In [ ]:
pot_MW_11 = agama.Potential("$potential_dir/vasiliev24/L2M11/potential_mw_init.ini")

In [ ]:
pot_EP = agama.Potential("$potential_dir/EP2020.ini")

In [ ]:
function density_axis(gs)
    return Axis(gs[1,1],
        xlabel=L"log $r$ / kpc",
        ylabel = L"$\log\,\bar\rho / 10^{10}M_\odot$\,kpc$^{-3}$",
        )
end


In [ ]:
function calc_ρ_mean(prof::LilGuys.SphericalProfile, r)
    y = LilGuys.calc_M.(prof, r)
    rho_mean = @. y / (4π/3*r^3)
end

In [ ]:
function calc_ρ_mean(prof::Py, r; vasiliev_units=false)
    rho = prof.enclosedMass(np.array(r))
    M = pyconvert(Array{Float64}, rho) 
    if vasiliev_units
        M .*= V_M2MSUN ./ M2MSUN
    end

    y = @. M / (4π/3 * r^3)
end

In [ ]:
function plot_ρ_mean!(ax, prof::LilGuys.SphericalProfile)
    x = LinRange(-2, 1.5, 1000)
    r = 10 .^ x
    y = calc_ρ_mean(prof, r,)

    lines!(x, log10.(y))
end

In [ ]:
function plot_acc!(ax, prof::LilGuys.SphericalProfile)
    x = LinRange(-2, 1.5, 1000)
    r = 10 .^ x
    y = LilGuys.calc_Φ.(prof, r)
    rho_mean = LilGuys.gradient(y, r)

    lines!(x, log10.(rho_mean ./ r))
end

In [ ]:
np = pyimport("numpy")

In [ ]:
function plot_ρ_mean!(ax, prof::Py; xlimits=(-2, 1.5), vasiliev_units=false, kwargs...)
    x = LinRange(xlimits[1], xlimits[2], 1000)
    r = 10 .^ x
 
    y = calc_ρ_mean(prof, r, vasiliev_units=vasiliev_units)
    lines!(x, log10.(y); kwargs...)
end

In [ ]:
function plot_acc!(ax, prof::Py; xlimits=(-2, 1.5), vasiliev_units=false, kwargs...)
    x = LinRange(xlimits[1], xlimits[2], 1000)
    r = np.array((10 .^ x) .* [0, 0, 1]')
    rho = prof.force(r)
    acc = pyconvert(Matrix{Float64}, rho)'
    a = calc_r(acc)
    r = 10 .^ x
    
    if vasiliev_units
        a .*= (V_M2MSUN ./ M2MSUN) ./ (V_T2GYR ./ T2GYR) .^ 2
    end

    lines!(x, log10.(a ./ r); kwargs...)
end

In [ ]:
LilGuys.arcmin_to_kpc(30, 83.2)

In [ ]:
sv = 9.6/ V2KMS
rh = 0.73
dlρ_mean = 0.2
ρ_mean = 5/3 * 3*sv^2 / (4π/3 * rh^2)

In [ ]:
-0.089*V_T2GYR

In [ ]:
50 * V_M2MSUN / M2MSUN * 3 .* (10 .^ (-dlρ_mean), 1, 10 .^ (dlρ_mean))

In [ ]:
V_T2GYR

In [ ]:
let
    fig = Figure()
    ax = density_axis(fig[1,1])

    plot_ρ_mean!(ax, scl_profile)

    hlines!(log10(ρ_mean), color=:black, label="Sculptor mean density")
    hspan!(log10(ρ_mean) - dlρ_mean, log10(ρ_mean) + dlρ_mean, color=(:black, 0.1))
    vlines!(log10(rh))
    axislegend(position=:lb)
    fig
end

In [ ]:
let
    fig = Figure()
    ax = density_axis(fig[1,1])

    plot_acc!(ax, scl_profile)
    hlines!(log10.(3sv^2 / rh^2))

    fig
end

In [ ]:
let
    fig = Figure()
    ax = density_axis(fig[1,1])

    plot_ρ_mean!(ax, pot_EP, xlimits=(0, 2), label="EP2020")
    plot_ρ_mean!(ax, pot_MW_10, xlimits=(0, 2), vasiliev_units=true, label="V24 M=10")
    plot_ρ_mean!(ax, pot_MW_11, xlimits=(0, 2), vasiliev_units=true, label="V24, M=11")

    hlines!(log10(ρ_mean/3), color=:black, label="1/3 sculptor")
    hspan!(log10(ρ_mean/3) - dlρ_mean, log10(ρ_mean/3) + dlρ_mean, color=(:black, 0.1))

    axislegend(position=:lb)
    fig
end

In [ ]:
let
    fig = Figure()
    ax = Axis(fig[1,1])

    plot_acc!(ax, pot_EP, xlimits=(0, 2), vasiliev_units=false, label="V+21, M=1.5")

    hlines!(log10.(sv^2 / rh^2))
    axislegend(position=:lb)
    ax
    fig
end

In [ ]:
let
    fig = Figure()
    ax = density_axis(fig[1,1])

    plot_ρ_mean!(ax, pot_lmc_1_5, xlimits=(-1, 2), vasiliev_units=true, label="V+21, M=1.5")
    plot_ρ_mean!(ax, pot_lmc_2, xlimits=(-1, 2), vasiliev_units=true, label="V24 M=2")
    plot_ρ_mean!(ax, pot_lmc_3, xlimits=(-1, 2), vasiliev_units=true, label="V24, M=3")

    hlines!(log10(ρ_mean/3), color=:black, label="1/3 sculptor")
    hspan!(log10(ρ_mean/3) - dlρ_mean, log10(ρ_mean/3) + dlρ_mean, color=(:black, 0.1))

    axislegend(position=:lb)
    ax
    fig
end

In [ ]:
log10(8.7), log10(1.7/ (8.7^3 * 4π/3))

In [ ]:
function calc_rperi_rj(host, satellite; rh=rh, limits=(0, 2), vasiliev_units=false)
    x = LinRange(limits[1], limits[2], 1000)

    r = 10 .^ x

    ρ_mean_host = LilGuys.lerp(r, calc_ρ_mean(host, r, vasiliev_units=vasiliev_units))

    r2 = 10 .^ LinRange(-2, 3, 1000)
    ρ_mean_sat = LilGuys.lerp(r2, calc_ρ_mean(satellite, r2))

    rj = zeros(length(r))

    rperi = r
    
    for i in eachindex(rj)

        rr = rperi[i]
        rj[i] = LilGuys.find_zero(r -> ρ_mean_sat(r) - 3*ρ_mean_host(rr), 1.)
    end

    return rperi, rj
end

    

In [ ]:
function plot_rperi_rj!(host, satellite; rh=rh, vasiliev_units=false, kwargs...)
    rp, rj = calc_rperi_rj(host, satellite; rh=rh, vasiliev_units=vasiliev_units)

    lines!(log10.(rp), log10.(rj); kwargs...)
end

In [ ]:
fig = Figure()
ax = Axis(fig[1,1];
    xlabel = "log pericentre / kpc",
    ylabel = "log rJ / kpc",
)

plot_rperi_rj!(pot_lmc_3, scl_profile, vasiliev_units=true, label="LMC host")
plot_rperi_rj!(pot_EP, scl_profile, label="Milky Way host")

vspan!(log10.(25), log10(32), label="periLMC / kpc", color=(COLORS[1], 0.2))
vspan!(log10.(45), log10(60), label="peri MW / kpc", color=(COLORS[2], 0.2))

hlines!(log10(rh), color=:black, label="tidal feature")
hspan!(log10(rh) - 0.05/0.25 / log(10), log10(rh) + 0.05/0.25 / log(10), color=(:black, 0.2))
axislegend(position=:lt)

fig

In [ ]:
rp, rj = calc_rperi_rj(pot_EP, scl_profile)

In [ ]:
rp[argmin(abs.(rj .- rh))]

In [ ]:
rp, rj = calc_rperi_rj(pot_lmc_3, scl_profile, vasiliev_units=true)

In [ ]:
rp[argmin(abs.(rj .- rh))]

In [ ]:
rp, rj = calc_rperi_rj(pot_lmc_2, scl_profile, vasiliev_units=true)

In [ ]:
rp[argmin(abs.(rj .- rh))]